# Interpretabilidad Mecanicista de Transformers

## Introducción

Este documento detalla el proceso para descargar e implementar el modelo **Llama 3.2 1B en fp16** desde **Hugging Face**, extrayendo la *n*-ésima salida de la capa MLP (*Multi-Layer Perceptron*). También se documentará cada paso siguiendo una metodología rigurosa.

## ¿Qué es Llama 3.2 1B?

Llama 3.2 es un modelo de lenguaje basado en la arquitectura **Transformer**, desarrollado por Meta. Su tamaño (1B de parámetros) lo hace eficiente para tareas de procesamiento del lenguaje natural.

## ¿Qué es fp16 y por qué es importante?

**fp16 (Floating Point 16 bits)** es un formato de precisión reducida que permite acelerar el entrenamiento y la inferencia del modelo, reduciendo el uso de memoria sin perder mucha precisión.

## ¿Qué es Hugging Face?

**Hugging Face** es una plataforma líder en el desarrollo de modelos de inteligencia artificial, especialmente en el campo del procesamiento del lenguaje natural (NLP). Proporciona una gran variedad de modelos preentrenados, herramientas para el entrenamiento y despliegue de modelos, y una comunidad activa de investigadores y desarrolladores.

### ¿Para qué se usa Hugging Face?

- **Repositorio de modelos:** Permite descargar y compartir modelos preentrenados.
- **Transformers Library:** Proporciona una API para usar modelos de NLP fácilmente.
- **Hugging Face Hub:** Un espacio para colaborar y almacenar modelos y datasets.
- **Inference API:** Para probar modelos sin necesidad de descargarlos localmente.

### Conceptos clave que usaremos en Hugging Face

1. **Token de autenticación:** Necesario para acceder a modelos privados o restringidos.
2. **Modelos preentrenados:** Conjuntos de pesos y configuraciones listos para usar.
3. **Tokenizer:** Convierte texto en tensores numéricos que el modelo puede procesar.
4. **Pipeline:** Interfaz sencilla para ejecutar modelos en tareas específicas.
5. **AutoModel y AutoTokenizer:** Clases que nos permiten cargar modelos y tokenizadores sin necesidad de conocer su arquitectura exacta.

---

## Acceso al modelo en Hugging Face

Para acceder a *Llama 3.2*, hay dos métodos principales:

### Método 1: Solicitud de acceso manual 

1. Ir a [Hugging Face](https://huggingface.co/).
2. Usar la barra de búsqueda para encontrar **Llama 3.2 1B**.
3. Llenar el formulario de solicitud de acceso.
4. Una vez aprobado, podrás acceder a los archivos del modelo en la pestaña **Files and versions**.

Este método es útil si el modelo tiene restricciones de acceso y no requiere autenticación en código.

### Método 2: Autenticación con un token de acceso

Si necesitas automatizar el proceso o descargar modelos privados, puedes generar un token de acceso:

1. Ve a **Settings > Access Tokens** en Hugging Face.
2. Genera un nuevo *token* con permisos de "read".
3. Usa el siguiente código para autenticarte:

```python
from huggingface_hub import login
login("TU_TOKEN_AQUI")  # Reemplaza con tu token
```

Este método es más formal y útil si trabajas en servidores o con varios modelos en diferentes proyectos.

---

## Configuración del entorno

Instalaremos las dependencias necesarias en nuestro entorno de Python:

```bash
pip install torch torchvision torchaudio transformers huggingface_hub
```

Verificamos la instalación ejecutando:

```bash
python3 -c "import torch; print(torch.__version__)"
```

Si el comando muestra un número de versión (`2.x.x`), significa que **PyTorch está correctamente instalado**.

Opcionalmente, podemos crear un entorno virtual (quizá esrte paso lo debas hacer primero para poder instalar paquetes, ya deoendera de tu gestor de paquetes que tengas en uso):

```bash
python -m venv llama_env
source llama_env/bin/activate  # En macOS/Linux
llama_env\Scripts\activate  # En Windows
```

---

## Descarga del modelo desde Hugging Face

Cargaremos el modelo y el *tokenizer* desde Hugging Face:

```python
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-3.2-1B"  # Nombre exacto del modelo

# Cargar el tokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print("Tokenizer cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el tokenizer: {e}")

# Cargar el modelo completo
try:
    model = AutoModelForCausalLM.from_pretrained(model_name)
    print("Modelo cargado exitosamente.")
except Exception as e:
    print(f"Error al cargar el modelo: {e}")
```

Si el modelo se descarga correctamente, estará listo para su análisis y uso en inferencias. En caso de errores, verifica los permisos de acceso en Hugging Face o la correcta instalación de las dependencias.

---

## Estructura del modelo Llama 3.2 1B

Llama 3.2 1B sigue la arquitectura **Transformer**, que está compuesta por múltiples bloques de procesamiento de información. Sus principales componentes son:

### 🔹 1. **Embeddings**
Los embeddings convierten palabras o tokens en vectores numéricos de alta dimensión. Estos vectores son la entrada del modelo y representan el significado de las palabras en un espacio matemático.

### 🔹 2. **Múltiples capas de atención (Self-Attention)**
Cada capa de atención analiza las relaciones entre todas las palabras de la oración para determinar cuáles son más relevantes para la predicción.

### 🔹 3. **Capa MLP (Multi-Layer Perceptron)**
Después de cada capa de atención, los datos pasan por una **MLP (Red Neuronal de Múltiples Capas)**. Esta capa:
   - Refina la información extraída por la atención.
   - Introduce no linealidad al modelo.
   - Generaliza mejor las representaciones del lenguaje.

Cada MLP en el Transformer tiene dos capas completamente conectadas con una función de activación no lineal intermedia.

### 🔹 4. **Capa de salida**
Finalmente, la capa de salida del modelo convierte la representación final en una probabilidad de predicción sobre el siguiente token en la secuencia.

### 🔹 5. **Estructura en profundidad**
Llama 3.2 1B tiene varias capas Transformer apiladas, donde cada capa tiene una subcapa de **self-attention** y una subcapa MLP. Las activaciones intermedias dentro de la MLP son esenciales para analizar la información que el modelo está aprendiendo en cada paso.

---

## Guardar y cargar el modelo localmente

Para evitar descargar el modelo cada vez que lo necesitemos, podemos almacenarlo localmente:

```python
# Guardar el modelo localmente
model.save_pretrained("./llama3_model")
tokenizer.save_pretrained("./llama3_model")

# Cargarlo más tarde sin conexión
from transformers import AutoModelForCausalLM, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("./llama3_model")
model = AutoModelForCausalLM.from_pretrained("./llama3_model")
```

Esto optimiza el tiempo y evita depender de internet en cada ejecución.

---

## Generar texto con Llama 3.2

Una vez que el modelo está cargado, podemos probarlo generando texto:

```python
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Definir el modelo
model_name = "meta-llama/Llama-3.2-1B"

# Cargar el tokenizer y el modelo
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Verificar si MPS está disponible y mover el modelo a GPU
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)

# Tokenizar la entrada y mover a GPU
prompt = "Explica la importancia de la interpretabilidad en transformers."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generar texto con MPS activado
print("Generando texto...")
with torch.no_grad():
    outputs = model.generate(
        **inputs, 
        max_length=100,  # Reducido para mejorar velocidad
        temperature=0.7,  # Controla la aleatoriedad
        top_p=0.9  # Controla la diversidad de respuestas
    )
print("Generación completada.")

# Decodificar la salida del modelo
texto_generado = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("Texto generado:", texto_generado)

```

Este código permite ver cómo Llama 3.2 responde a una consulta en lenguaje natural.


# Explorando la arquitectura interna de Llama 3.2 1B

Antes de extraer salidas intermedias, es fundamental comprender cómo está estructurado el modelo.

## 1️⃣ Ver la configuración general del modelo

La configuración del modelo contiene información clave como:
- Número de capas (`num_hidden_layers`)
- Dimensión de los embeddings (`hidden_size`)
- Número de cabezas de atención (`num_attention_heads`)
- Tamaño de la capa intermedia MLP (`intermediate_size`)

```python
from transformers import AutoModelForCausalLM

model_name = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(model_name)

# Ver configuración general del modelo
print(model.config)
```

### Explicación detallada de la configuración del modelo:

- **_name_or_path**: indica el nombre exacto del modelo que estamos utilizando.
- **architectures**: muestra la clase principal usada para la arquitectura, en este caso `LlamaForCausalLM`.
- **attention_bias**: especifica si hay un sesgo aplicado en las matrices de atención (aquí es `false`).
- **attention_dropout**: nivel de abandono (dropout) aplicado en la atención (0.0 significa sin dropout).
- **bos_token_id / eos_token_id**: identificadores especiales de inicio y fin de secuencia.
- **head_dim**: tamaño de la dimensión de cada cabeza de atención.
- **hidden_act**: función de activación en la MLP (aquí `silu`).
- **hidden_size**: tamaño de la representación oculta, es decir, la dimensión del embedding (2048).
- **initializer_range**: rango usado para inicializar los pesos.
- **intermediate_size**: tamaño de la capa intermedia de la MLP (8192).
- **max_position_embeddings**: el máximo número de posiciones que puede procesar el modelo (131072).
- **mlp_bias**: indica si la MLP usa sesgos (falso en este caso).
- **model_type**: tipo de modelo (`llama`).
- **num_attention_heads**: número de cabezas de atención (32).
- **num_hidden_layers**: número de capas Transformer (16).
- **num_key_value_heads**: número de cabezas para valores y llaves (8).
- **pretraining_tp**: indica partición para entrenamiento (1 significa sin particiones).
- **rms_norm_eps**: valor epsilon para estabilidad numérica en normalización.
- **rope_scaling**: contiene parámetros relacionados con el mecanismo de rotación posicional (RoPE).
- **rope_theta**: parámetro adicional para la frecuencia rotacional (500000.0).
- **tie_word_embeddings**: indica si las embeddings de entrada y salida están ligadas.
- **torch_dtype**: tipo de datos usado (`float32`).
- **transformers_version**: versión de la librería Transformers usada (4.49.0).
- **use_cache**: indica si se utiliza cache para acelerar inferencias.
- **vocab_size**: tamaño del vocabulario (128256 tokens).

---

## 2️⃣ Ver cuántos bloques Transformer tiene el modelo

El modelo tiene una lista de capas accesible con `model.model.layers`. Cada elemento es un bloque Transformer completo.

```python
# Visualizar la lista de bloques Transformer
print(model.model.layers)
```

### Estructura de las capas:

La salida muestra una `ModuleList` con 16 bloques (`0-15`), cada uno compuesto por:

- **LlamaAttention**: contiene 4 proyecciones lineales:
  - `q_proj` (queries), `k_proj` (keys), `v_proj` (values), y `o_proj` (output).
- **mlp (LlamaMLP)**: contiene tres capas lineales:
  - `gate_proj` y `up_proj` (ambas proyectan de 2048 a 8192 dimensiones), y `down_proj` (reduce de 8192 a 2048).
  - Una función de activación **SiLU()**.
- **input_layernorm y post_attention_layernorm**: normalizaciones RMS aplicadas antes y después de la atención.

Ejemplo de bloque Transformer impreso:

```python
ModuleList(
  (0-15): 16 x LlamaDecoderLayer(
    (self_attn): LlamaAttention(
      (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
      (k_proj): Linear(in_features=2048, out_features=512, bias=False)
      (v_proj): Linear(in_features=2048, out_features=512, bias=False)
      (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
    )
    (mlp): LlamaMLP(
      (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
      (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
      (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
      (act_fn): SiLU()
    )
    (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
    (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
  )
)
```

---

## 3️⃣ Analizar un bloque Transformer específico

Para entender mejor la estructura interna, accedemos al primer bloque (índice 0):

```python
# Acceder al primer bloque Transformer
primer_bloque = model.model.layers[0]

# Ver la estructura interna del primer bloque
print(primer_bloque)
```

---

## 4️⃣ La MLP dentro de un bloque Transformer

Cada bloque Transformer contiene un submódulo `mlp` que es un perceptrón multicapa con dos capas lineales y una activación no lineal.

```python
# Visualizar la estructura de la MLP dentro del primer bloque
print(primer_bloque.mlp)
```

---

## ✅ Resumen hasta aquí:
- El modelo Llama 3.2 1B tiene varios bloques Transformer.
- Cada bloque contiene un módulo de autoatención y una MLP.
- La MLP procesa y refina la información extraída por la autoatención.
- La comprensión de esta estructura es clave antes de extraer la salida de la MLP.

En la siguiente sección veremos cómo interceptar y extraer la salida de la MLP para la capa que deseemos (la *n*-ésima salida).
